In [1]:
import os 
import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt  
import matplotlib.colors as col

from pdpbox import pdp
from matplotlib import cm
from pdpbox import info_plots
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import roc_auc_score
from matplotlib.pyplot import MultipleLocator

plt.rc('font',family='Arial')
plt.rcParams ['svg.fonttype'] ='none'
plt.rcParams ['svg.fonttype'] ='none'
pd.options.mode.chained_assignment = None
%matplotlib inline

### load data

In [10]:
#train 
train = pd.read_csv(r'../../data/west/train.csv',index_col=0)
train['month'] = train['date'].apply(lambda x:pd.Timestamp(x).month)
train = train.drop(columns=['date','lon','lat','Open Water','Urban-Built-up','Snow-Ice'])
X_train=train.iloc[:,1:]
y_train=train.iloc[:,0]
#test 
test = pd.read_csv(r'../../data/west/test.csv',index_col=0)
test['month'] = test['date'].apply(lambda x:pd.Timestamp(x).month)
test = test.drop(columns=['date','lon','lat','Open Water','Urban-Built-up','Snow-Ice'])
X_test=test.iloc[:,1:]
y_test=test.iloc[:,0]

### train model

In [ ]:
xgb_model = XGBClassifier(
                  n_jobs=4,
                  max_depth=8,
                  learning_rate=0.03,
                  n_estimators=300,
                  verbosity=1,
                  objective='binary:logistic',
                  booster='gbtree',
                  gamma=0.4,
                  min_child_weight=4,
                  subsample=0.9,
                  colsample_bytree=0.8,
                  reg_alpha=0.1,
                  reg_lambda=1,
                  base_score=0.5,
                  eval_metric='auc',
              )
xgb_model.fit(X_train, y_train,eval_set=[(X_test, y_test)],verbose=True)

### varibale importance

In [32]:
imp_dict = xgb_model.get_booster().get_score(importance_type='weight')
model_importance = pd.DataFrame([imp_dict.keys(),imp_dict.values()]).T
model_importance.columns=['Variable','Weight']
model_importance['Weight'] = model_importance.Weight.apply(lambda x:x/sum(model_importance.Weight))
model_importance = model_importance.sort_values(by='Weight',ascending=False)
model_importance.to_csv(r'../../result/figure_data/variable_importance/west.csv')

### generate partial dependence plot by pdpbox
#### Calculate 2d partial dependence plot 

In [22]:
data_features = list(X_train.columns)
pdp_month_dL = pdp.pdp_interact(model=xgb_model,
                                dataset=train,
                                model_features=data_features,
                                features=['month', 'distance_light'],
                                num_grid_points=[12, 11],
                                grid_types = ['equal','percentile'],
                                n_jobs=1)

In [25]:
pdp_month_dL.feature_grids[0]
(pdp_month_dL.feature_grids[1])

array([0.00000000e+00, 5.43671652e-02, 2.17739955e+00, 4.66250627e+00,
       8.84273668e+00, 1.50846849e+01, 2.40062055e+01, 3.67489868e+01,
       5.49981314e+01, 1.00333032e+02, 4.31023533e+02])

In [26]:
data_features = list(X_train.columns)
pdp_month_dL = pdp.pdp_interact(model=xgb_model,
                                dataset=train,
                                model_features=data_features,
                                features=['month', 'distance_light'],
                                num_grid_points=[12, 11],
                                grid_types = ['equal','percentile'],
                                n_jobs=1)
pdp_month_dL = pdp.pdp_interact(model=xgb_model,
                                dataset=train,
                                model_features=data_features,
                                features=['month', 'distance_light'],
                                num_grid_points=[12, 11],
                                grid_types = ['equal','percentile'],
                                ##grid[1].astype(int) will ingnore 0.5,and generate list which length=10, so the grid[1] was inputted manually
                                cust_grid_points=[pdp_month_dL.feature_grids[0],[0,0.5,2,5,9,15,24,37,55,100,431]],
                                #percentile_ranges=[ None,(0, 95)],
                                n_jobs=1)
pdp_v = pdp_month_dL.pdp.preds.values
pdp_v.resize(12,11)
pd_pdp = pd.DataFrame(pdp_v).T
pd_pdp.columns = pdp_month_dL.feature_grids[0].astype(int)
pd_pdp['Distance to light'] = pdp_month_dL.feature_grids[1]
pd_pdp = pd_pdp.set_index('Distance to light')
pd_pdp.to_csv(r'../../result/figure_data/2d_pdp/figure_2a_west.csv')

#### Generate the figure2a

In [ ]:
file = r'../../result/figure_data/2d_pdp/figure_2a_west.csv'
df = pd.read_csv(file).sort_values(by='Distance to light',ascending=False)
df['Distance to light'] = df.apply(lambda x:round(x['Distance to light'],1),axis=1)
df = df.T
df.columns=df.iloc[0,:]
df = df.drop(index='Distance to light')
df = df[sorted(df.columns)]
grid_kws = {"width_ratios": (0.9, 0.06)}  # heatmap:colorbar = 0.9：0.1

fig, (ax, cbar_ax)  = plt.subplots(1, 2,gridspec_kw=grid_kws,figsize=(10,10)) 

sns.heatmap(data=df,vmin=0.38,vmax=0.62,cmap='RdYlBu_r',linewidths=0.1,robust=False,
                square=False,ax=ax,cbar_ax=cbar_ax,cbar_kws={"orientation": "vertical"})
ax.set_xticklabels([0,0.5,2,5,9,15,24,37,55,100,431],fontdict={'size':22})
ax.set_yticklabels(['January','February','March','April','May','June','July',
                    'August','September','October','November','December'],rotation=0,fontdict={'size':26})

ax.set_xlabel("Distance to light (km)",fontdict={'size':26},labelpad=10)
cbar_ax.set_yticklabels((cbar_ax.get_yticklabels()),fontdict={'size':22})
ax.set_title("Western China",fontdict={'size':32},pad=25)

plt.subplots_adjust(left = None,bottom = None,right =None,top = None,wspace = 0.1,hspace = None)
plt.savefig(r'../../result/figure/2d_pdp/figure_2a_west.pdf',dpi=300,bbox_inches = 'tight')
plt.savefig(r'../../result/figure/2d_pdp/figure_2a_west.png',dpi=300,bbox_inches = 'tight')

#### Calculate 1d partial dependence plot for every vaibale

In [29]:
def par_dep(xs, frame, model, resolution=25, bins=[None]):   
    pd.options.mode.chained_assignment = None
    par_dep_frame = pd.DataFrame(columns=[xs, 'partial_dependence'])
    col_cache = frame.loc[:, xs].copy(deep=True)
    if bins[0] == None:
        min_ = frame[xs].min()
        max_ = frame[xs].max()
        by = (max_ - min_)/resolution

        bins = np.arange(min_, max_, by)
        print(bins)
    for j in bins:
        frame.loc[:, xs] = j
        #dframe = xgb.DMatrix(frame)
        par_dep_i = pd.DataFrame(model.predict_proba(frame)[:,1])
        par_dep_j = par_dep_i.mean()[0]
        par_dep_frame = par_dep_frame.append({xs:j,
                                              'partial_dependence': par_dep_j}, 
                                              ignore_index=True)
    frame.loc[:, xs] = col_cache
    return par_dep_frame
        
def hist(xs, frame,bins):   
    pd.options.mode.chained_assignment = None
    hist_list = []
    
    for i in range(len(bins)-1):
        temp = len(frame[xs][(frame[xs]>=bins[i]) & (frame[xs]<bins[i+1])])
        hist_list.append([bins[i],temp/len(frame[xs])])
        
    temp = len(frame[xs][(frame[xs]>=bins[-1])])
    hist_list.append([bins[-1],temp/len(frame[xs])])
    hist_df = pd.DataFrame(hist_list,columns=[xs,'hist'])
    return hist_df

## 输出所有的hist和pdp，输入X_total和model
def out_pdp_hist(par_dataset,xgb_model,V_bins,save_path =None,variables=None):
    if save_path==None:
        save_path=r'../../result/figure_data/1d_pdp/'
    if variables==None:
        variables=['Barren', 'Cultivated and Managed Vegetation',
                    'Deciduous Broadleaf Trees', 'Evergreen Broadleaf Trees',
                    'Evergreen Deciduous Needleleaf Trees', 'Herbaceous Vegetation',
                    'Mixed-Other Trees', 'Regularly Flooded Vegetation', 'Shrubs',
                    'Snow-Ice', 'distance_light', 'distance_water', 'Temperature',
                    'U_Wind', 'V_wind', 'Precipitation', 'NDVI','Pop','month','year_precipitation']
    #X = par_dataset[variables]
    for v in variables:
        #print(v)
        plot_bin = V_bins[v]
        par_dep_v = par_dep(v,par_dataset, xgb_model,bins=plot_bin)
        hist_v = hist(v,par_dataset,bins=plot_bin)
        if v=='Air_pressure':
            v='Surface Pressure'
        par_dep_v.to_csv(save_path+'/pdp_csv/%s_%s.csv'%('pdp',v),index=False)
        hist_v.to_csv(save_path+'/hist_csv/%s_%s.csv'%('hist',v),index=False) 
        print(v)

In [ ]:
# 计算一套V_bins 
V_bins ={}
resolution = 20
for i in X_train.columns:
    if i=='month':
        V_bins[i]=np.arange(1,13,1)
    else:
        max_ = X_train[i].max()
        min_ = X_train[i].min()
        by = (max_ - min_)/resolution
        V_bins[i] = np.append(np.arange(min_, max_, by),max_)
        
my_bins = V_bins
my_save = r'../../result/figure_data/1d_pdp/west/'
my_variables = X_train.columns.to_list()
out_pdp_hist(par_dataset=X_train,xgb_model=xgb_model,V_bins=my_bins,save_path=my_save,variables=my_variables)

#### Generate the Extended Data Figure 11-12